In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/home/yiftach/main/Research/shap-e/shap_e/examples/shap_e_model_cache
None is the location


/home/yiftach/anaconda3/envs/instantmesh/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/yiftach/anaconda3/envs/instantmesh/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
100%|███████████████████████████████████████| 890M/890M [00:52<00:00, 17.8MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

None is the location


In [5]:
batch_size = 4
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("example_data/corgi.png")
image.shapze


AttributeError: 'PngImageFile' object has no attribute 'shape'

In [ ]:

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

In [5]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 256 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAPXw6/Pu6PLs5/Hp4uzo4evl3uzh1uPg2uHd1+vZxODb1N7a1N…

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAPPx7ujm4+Tj4ejc1N7d2+TWy+LPwtjTz9fMxdnIv9nGtdi9od…

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAPjr6Pfq6Pbp5/Xo5vTn5fPl4/Dl4u/j4e7i3+7f2+je2ufc2O…

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAPnw6Pjt5vfs5fbr4/Xq4/Pp4fTo4fHn3/Hl3u/l3u7k3O/j3O…